In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext
import numpy as np
import time

# Variables globales con información para acelerar procesamiento
g_filas = g_cols = 0 # Serán números (1.000.000 y 11 en este caso)

ITER = 40
APRN = 0.5

In [2]:
# linea es un string de la forma: "n0, n1, n2, ..., n10, r" donde:
#  nX es una propiedad del tráfico
#  r  es su clasificación (1 si es botnet, 0 si no lo es)
def formato_inicial(linea):
    datos = np.asarray([float(i) for i in linea[:-1]])
    resul = int(linea[-1])
    return (datos, resul)

# Devuelve un RDD de los datos del fichero fn.
# Cada registro del RDD es una tupla (X, y)
#  X: np.array de los datos.
def readFile(fn):
    global g_filas, g_cols
    sc      = SparkContext("local[*]", "BotnetParalelaPruebas")
    todo    = sc.textFile(fn).map(lambda fila: fila.split(','))
    ret     = todo.map(formato_inicial)
    g_filas = todo.count()
    g_cols  = len(ret.take(1)[0][0])
    return ret

t1  = time.time()
rdd = readFile("../../../datos/botnet_tot_syn_l.csv")
t2  = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))

Tiempo transcurrido: 4.134044408798218 segundos.


In [3]:
# rdd: RDD de 1.000.000 de registros (X, y)
#  X es un array con 11 flotantes
#  y es 1 ó 0
# Devuelve un np.array con las medias de las 11 columnas
def calcular_medias(rdd):
    global g_filas
    medias = np.asarray(rdd.reduce(lambda x, y: x + y)[0])/g_filas
    return medias

# rdd: RDD de 1.000.000 de registros (X, y)
#  X es un array con 11 flotantes
#  y es 1 ó 0
# medias: np.array de las medias de las 11 columnas
def calcular_stdev(rdd, medias):
    global g_filas
    parcial  = rdd.map(lambda x: (x[0]-medias)**2).reduce(lambda x, y: x + y)
    varianza = parcial/g_filas
    stdev    = np.sqrt(varianza)
    return stdev

# Toma por parámetro un RDD de 1.000.000 de registros.
# Cada registro tiene la forma (X, y)
#  X es un array con 11 flotantes
#  y es 1 ó 0
# Devuelve un RDD equivalente donde la X de cada tupla está
# reescalada a N(0, 1) (media = 0, desv. est. = 1)
def normalizar(rdd):
    t1 = time.time()
    medias = calcular_medias(rdd)
    t2 = time.time()
    stdevs = calcular_stdev(rdd, medias)
    t3 = time.time()
    normal = rdd.map(lambda x: ((x[0] - medias)/stdevs, x[1]))
    t4 = time.time()
    print("Tiempo medias: {} s.".format(t2 - t1))
    print("Tiempo stdevs: {} s.".format(t3 - t2))
    print("Tiempo normal: {} s.".format(t4 - t3))
    return normal
    

t1 = time.time()
datos = normalizar(rdd)
t2 = time.time()
print("Tiempo transcurrido: {} segundos.".format(t2 - t1))
# La media tiene que ser ~0
# La desviación estándar tiene que ser ~1

# 1.2345e-05 = 0.000012345

Tiempo medias: 83.86522245407104 s.
Tiempo stdevs: 4.37181830406189 s.
Tiempo normal: 1.1205673217773438e-05 s.
Tiempo transcurrido: 88.23725152015686 segundos.


In [4]:
comprobar_medias = calcular_medias(datos)
comprobar_stdevs = calcular_stdev(datos, comprobar_medias)
print(" +------------------------------- MEDIAS (~0) -----------------------------------+")
print(comprobar_medias)
print(" +------------------------------- STDEVS (~1) -----------------------------------+")
print(comprobar_stdevs)

 +------------------------------- MEDIAS (~0) -----------------------------------+
[4.38670935e-09 5.20124649e-08 1.23550534e-09 6.74973426e-12
 2.34923008e-13 1.49440578e-12 1.23479691e-06 1.06153422e-06
 1.29524385e-06 1.09851433e-06 9.45998204e-07]
 +------------------------------- STDEVS (~1) -----------------------------------+
[1.         0.99999997 1.         1.         1.         1.
 0.99999893 0.99999929 0.99999895 0.99999896 0.99999918]


In [5]:
def sigm(fila, pesos, sesgo):
    entrada = sesgo + np.sum(fila * pesos)
    p = 1/(1 + np.exp(-(entrada)))
    return p

def calcula_derivpesos(rdd, pesos, sesgo):
    parcial = rdd.map(lambda x: x[0]*(sigm(x[0], pesos, sesgo)-x[1]))
    suma    = parcial.reduce(lambda x,y: x+y)
    dp      = suma/g_filas
    return dp

def calcula_derivsesgo(rdd, pesos, sesgo):
    parcial = rdd.map(lambda x: sigm(x[0], pesos, sesgo)-x[1])
    suma    = parcial.reduce(lambda x,y: x+y)
    db      = suma/g_filas
    return db

def calcula_coste(rdd, pesos, sesgo):
    parcial = rdd.map(lambda x: (x[1]*np.log(sigm(x[0], pesos, sesgo)))+((1-x[1])*np.log(1-sigm(x[0], pesos, sesgo))))
    coste   = parcial.reduce(lambda x,y: x+y)
    coste = -(1/g_filas) * coste
    return coste

def entrenar(datos, iteraciones, aprendizaje):
    np.random.seed(6)
    pesos  = np.random.random([g_cols, ])
    dpesos = np.zeros([g_cols, ])
    sesgo  = 0.0
    dsesgo = 0.0
    t1 = time.time()
    for it in range(iteraciones):
        it_i = time.time()
        # Calcula las derivadas de los pesos
        dpesos = calcula_derivpesos(datos, pesos, sesgo)
        # Calcula la derivada del sesgo
        dsesgo = calcula_derivsesgo(datos, pesos, sesgo)
        # Reasigna los pesos y el sesgo
        pesos = pesos - dpesos * aprendizaje
        sesgo = sesgo - dsesgo * aprendizaje
        # Calcula el coste
        coste = calcula_coste(datos, pesos, sesgo)
        it_f = time.time()
        print("{}. coste: {:.4f} [{:.2f} segundos]".format(it, coste, (it_f - it_i)))
    
    t2 = time.time()
    print("Tiempo total: {:.2f}".format(t2 - t1))
    print("Media por iteración: {:.2f}".format((t2 - t1)/iteraciones))
    
    return pesos, sesgo

pesos, sesgo = entrenar(datos, ITER, APRN)

0. coste: 1.1418 [24.56 segundos]
1. coste: 0.7225 [24.90 segundos]
2. coste: 0.5858 [25.29 segundos]
3. coste: 0.5311 [24.56 segundos]
4. coste: 0.4933 [24.71 segundos]
5. coste: 0.4625 [25.86 segundos]
6. coste: 0.4368 [25.42 segundos]
7. coste: 0.4151 [24.85 segundos]
8. coste: 0.3965 [25.28 segundos]
9. coste: 0.3804 [24.73 segundos]
10. coste: 0.3665 [24.75 segundos]
11. coste: 0.3542 [24.66 segundos]
12. coste: 0.3434 [24.52 segundos]
13. coste: 0.3337 [25.19 segundos]
14. coste: 0.3251 [25.86 segundos]
15. coste: 0.3173 [25.35 segundos]
16. coste: 0.3102 [25.67 segundos]
17. coste: 0.3038 [25.37 segundos]
18. coste: 0.2979 [25.13 segundos]
19. coste: 0.2925 [24.75 segundos]
20. coste: 0.2875 [24.79 segundos]
21. coste: 0.2829 [25.43 segundos]
22. coste: 0.2786 [25.02 segundos]
23. coste: 0.2746 [24.86 segundos]
24. coste: 0.2709 [25.00 segundos]
25. coste: 0.2674 [24.85 segundos]
26. coste: 0.2642 [24.91 segundos]
27. coste: 0.2611 [24.88 segundos]
28. coste: 0.2582 [26.80 segun

In [6]:
def precision(datos, pesos, sesgo):
    paso1 = datos.map(lambda x: (sigm(x[0], pesos, sesgo), x[1]))
    paso2 = paso1.map(lambda x: (np.rint(x[0]) == x[1], 1))
    preds = paso2.reduceByKey(lambda x, y: x + y)
    
    preci = preds.collectAsMap()[True]/g_filas
    return preci

t1 = time.time()
precision_ = precision(datos, pesos, sesgo)
t2 = time.time()
print("La precisión es: {} [{}]".format(precision_, (t2 - t1)))

La precisión es: 0.932038 [8.507985591888428]
